In [ ]:
from functools import reduce  # python3 compatibility
from operator import mul
import json
import os
import statistics
import itertools
import math
import numpy as np
import itertools
import numpy as np
import time
from dataclasses import dataclass
from enum import Enum
import numpy as np
import random


In [ ]:


def gen_tasks(task_num, max_capNum, capabilities):
    """
    Generate tasks, each task is represented by a list of capabilities it requires
    :param: `task_num`: the number of tasks
    :param: `max_capNum`: the maximum number of capabilities a task could require
    :param: `capabilities`: the list of capabilities
    :return: the list of tasks. Each task is represented by a list of capabilities it requires.
    """
    # n is the number of task, max_capNum is the maximum number of cap a task could require
    return [
        sorted(
            np.random.choice(
                a=capabilities, size=np.random.randint(3, max_capNum + 1), replace=False
            )
        )
        for j in range(0, task_num)
    ]


def gen_constraints(agent_num, task_num, power=1, a_min_edge=2, t_max_edge=5):
    """
    Generate agent's constraints, each agent is represented by a list of tasks it has capability to work on.
    :param: `agent_num`: the number of agents
    :param: `task_num`: the number of tasks
    :param: `power`: the power used to magnify the probability
    :param: `a_min_edge`: the minimum number of tasks an agent has the capabilities work on.
    :param: `t_max_edge`: the maximum number of agents that could work on a task.
    :return: For each agent, the list of tasks it has the capabilities to work on. For each task, the list of agents that could work on it.
    """

    # power is the inforce you put in the probabilities
    # the maximum tasks an agent could work on depends on the number of tasks available (e.g, if |T| = 1/2|A|, then roughly each agent can work on two tasks)

    # calculate the max and min edges for agents
    available_seats = math.floor(t_max_edge * task_num)
    a_taskInds = [[] for i in range(0, agent_num)]
    a_taskNums = []
    for i in range(0, agent_num):
        a_max_edge = min((available_seats - a_min_edge * (agent_num - 1 - i)), t_max_edge, task_num)
        a_min_edge = min(a_min_edge, a_max_edge)
        
        # radomly indicate the number of task the agent could work on, based on the maximum and minimum number of tasks the agent could work on
        a_taskNum = np.random.randint(a_min_edge, a_max_edge + 1)
        
        a_taskNums.append(a_taskNum)
        
        available_seats -= a_taskNum

    t_agents_counts = [0 for j in range(0, task_num)]  # each indicate the current number of agents on the task

    # make sure no further draw for those reached the maximum limit.
    t_indexes = [j for j in range(0, task_num) if t_agents_counts[j] < t_max_edge]

    for i, a_taskNum in enumerate(a_taskNums):
        if any(tc == 0 for tc in t_agents_counts):  # if there are tasks that have not been allocated to any agent
            t_prob = [
                (math.e ** (t_max_edge - t_agents_counts[j])) ** power
                for j in t_indexes
            ]  # power is used to manify the probability
            sum_prob = sum(t_prob)
            t_prop_2 = [prop / sum_prob for prop in t_prob]

            # draw tasks accounting to their current allocations
            a_taskInds[i] = list(
                np.random.choice(
                    a=t_indexes,
                    size=min(a_taskNum, len(t_indexes)),
                    replace=False,
                    p=[prop / sum_prob for prop in t_prob],
                )
            )
            # increase the chosen task counters
        else:
            a_taskInds[i] = list(
                np.random.choice(
                    a=t_indexes, size=min(a_taskNum, len(t_indexes)), replace=False
                )
            )

        for j in a_taskInds[i]:
            t_agents_counts[j] += 1

        # make sure no further draw for those reached the maximum limit.
        t_indexes = [
            j for j in range(0, task_num) if t_agents_counts[j] < t_max_edge
        ]

    # get also the list of agents for each task
    t_agents = [
        [i for i in range(0, agent_num) if j in a_taskInds[i]]
        for j in range(0, task_num)
    ]

    return a_taskInds, t_agents


def gen_agents(a_taskInds, tasks, max_capNum, capabilities, max_capVal):  
    # m is the number of task, max_capNum is the maximum number of cap a task could require, max_capVal is the maximum capability value
    """
    Generate agents, each agent is represented by a list of capabilities it has and a list of contribution values for each capability
    :param: `a_taskInds`: the list of list of tasks each agent could work on
    :param: `tasks`: the list of tasks, represented by a list of capabilities it requires
    :param: `max_capNum`: the maximum number of capabilities an agent could have
    :param: `capabilities`: the list of capabilities
    :param: `max_capVal`: the maximum value of a capability
    """
    caps_lists = []
    contri_lists = []
    for a_taskInd in a_taskInds:
        t_caps = [tasks[j] for j in a_taskInd]  # lists of caps that each task agent could perform

        caps_union = set(itertools.chain(*t_caps))  # union of unique caps of tasks that agent could perform.

        a_cap_num = np.random.randint(
            min(3, max_capNum, len(caps_union)), 
            min(len(caps_union), max_capNum) + 1
        )  # the num of caps the agent will have

        a_caps = set([np.random.choice(t_c) for t_c in t_caps])  # initial draw to guarantee the agent has some contribution to each of the task that the agent has the capability to perform.

        # Randomly draw the remaining capabilities, possibly none
        remaining_choices = list(caps_union.difference(a_caps))
        if remaining_choices != []:
            a_caps.update(
                np.random.choice(
                    remaining_choices,
                    min(max(0, a_cap_num - len(a_taskInd)), len(remaining_choices)),
                    replace=False,
                )
            )
        
        # a_caps.update(np.random.choice(remaining_choices, min(0,len(remaining_choices),a_cap_num-len(a_taskInd)),replace = False))

        caps_list = sorted(list(a_caps))
        contri_list = [
            (np.random.randint(1, max_capVal + 1) if c in caps_list else 0)
            for c in range(0, len(capabilities))
        ]

        caps_lists.append(caps_list)
        contri_lists.append(contri_list)

    return caps_lists, contri_lists


def gen_agents_random(capabilities, agent_num, max_capNum, max_capVal):
    """
    Generate agents, each agent is represented by a list of capabilities it has and a list of contribution values for each capability
    :param: `capabilities`: the list of capabilities
    :param: `agent_num`: the number of agents
    :param: `max_capNum`: the maximum number of capabilities an agent could have
    :param: `max_capVal`: the maximum value of a capability
    """
    caps_lists = []
    contri_lists = []
    for i in range(0, agent_num):
        a_cap_num = np.random.randint(1, max_capNum + 1)  # the num of caps the agent will have
        a_caps = set(np.random.choice(capabilities, a_cap_num, replace=False))
        caps_list = sorted(list(a_caps))
        contri_list = [
            (np.random.randint(1, max_capVal + 1) if c in caps_list else 0)
            for c in range(0, len(capabilities))
        ]

        caps_lists.append(caps_list)
        contri_lists.append(contri_list)

    return caps_lists, contri_lists



def get_constraints(agents, tasks):
    """
    Calculate the constraints of the system, where the system consists of tasks and agents with constraints.

    :param: `agents`: the list of agents
    :param: `tasks`: the list of tasks
    :return: For each agent, the list of tasks it has the capabilities to work on. For each task, the list of agents that could work on it.
    """
    a_taskInds = []
    t_agentInds = []
    for agent in agents:
        a_taskInds.append([j for j, task in enumerate(tasks) if set(task) <= set(agent)])
    for task in tasks:
        t_agentInds.append([i for i, agent in enumerate(agents) if set(task) <= set(agent)])
    return a_taskInds, t_agentInds


In [ ]:
def task_reward(task, agents, gamma=1):
    # task is represented by a list of capabilities it requires, agents is a list agents, where each represented by a list cap contribution values
    """
    Calculate the reward of a single task
    :param: `task`: the list of capabilities the task requires
    :param: `agents`: the list of agents
    :param: `gamma`: the discount factor
    :return: the reward of the task
    """
    if agents == []:
        return 0
    else:
        return sum([max([agent[c] for agent in agents]) for c in task]) * (
            gamma ** len(agents)
        )


def sys_reward_agents(agents, tasks, allocation_structure, gamma=1):
    """
    Calculate the reward of the system, given the allocation structure: agent -> task
    """
    # allocation_structure is a vector of size M, each element indicate which task the agent is allocated to
    return sum(
        task_reward(task, [agent for i, agent in enumerate(agents) if allocation_structure[i] == j], gamma)
        for j, task in enumerate(tasks)
    )


def sys_rewards_tasks(tasks, agents, coalition_structure, gamma=1):
    """
    Calculate the reward of the system, given the coalition structure: task -> agents (coalition)
    """
    return sum(
        task_reward(task, [agents[i] for i in coalition_structure[j]], gamma)
        for j, task in enumerate(tasks)
    )





In [ ]:



def agent_contribution(agents, tasks, query_agentIndex, query_taskIndex, coalition, constraints, gamma=1):
    """
    Return contribution of agent i to task j in coalition C_j
    
    = U_i(C_j, j) - U_i(C_j \ {i}, j) if i in C_j

    = U_i(C_j U {i}, j) - U_i(S, j) if i not in C_j
    """
    a_taskInds = constraints[0]
    if query_taskIndex == len(tasks):
        return 0
    if query_taskIndex not in a_taskInds[query_agentIndex]:
        return 0
    cur_reward = task_reward(tasks[query_taskIndex], [agents[i] for i in coalition], gamma)
    if query_agentIndex in coalition:
        return cur_reward - task_reward(tasks[query_taskIndex], [agents[i] for i in coalition if i != query_agentIndex], gamma)
    else:
        return task_reward(tasks[query_taskIndex], [agents[i] for i in coalition] + [agents[query_agentIndex]], gamma) - cur_reward


def eGreedy2(agents, tasks, constraints, eps=0, gamma=1, coalition_structure=[]):
    re_assign = 0
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)
    allocation_structure = [task_num for i in range(0, agent_num)]  # each indicate the current task that agent i is allocated to, if = N, means not allocated
    if coalition_structure == []:
        coalition_structure = [[] for j in range(0, task_num + 1)]  # current coalition structure, the last one is dummy coalition
        cur_con = [0 for j in range(0, agent_num)]
    else:
        coalition_structure.append([])
        for j in range(0, task_num):
            for i in coalition_structure[j]:
                allocation_structure[i] = j
        cur_con = [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            for i, j in enumerate(allocation_structure)
        ]

    task_cons = [
        [
            agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
            if j in a_taskInds[i]
            else -1000
            for j in range(0, task_num)
        ] + [0]
        for i in range(0, agent_num)
    ]
    # the last 0 indicate not allocated

    move_vals = [
        [
            task_cons[i][j] - cur_con[i] if j in a_taskInds[i] + [task_num] else -1000
            for j in range(0, task_num + 1)
        ]
        for i in range(0, agent_num)
    ]

    max_moveIndexs = [
        np.argmax([move_vals[i][j] for j in a_taskInds[i]] + [0])
        for i in range(0, agent_num)
    ]

    max_moveVals = [
        move_vals[i][a_taskInds[i][max_moveIndexs[i]]]
        if max_moveIndexs[i] < len(a_taskInds[i])
        else move_vals[i][task_num]
        for i in range(0, agent_num)
    ]

    iteration = 0
    while True:
        iteration += 1
        feasible_choices = [i for i in range(0, agent_num) if max_moveVals[i] > 0]
        if feasible_choices == []:
            break  # reach NE solution
        # when eps = 1, it's Random, when eps = 0, it's Greedy
        if np.random.uniform() <= eps:
            # exploration: random allocation
            a_index = np.random.choice(feasible_choices)
        else:
            # exploitation: allocationelse based on reputation or efficiency
            a_index = np.argmax(max_moveVals)
            
        t_index = a_taskInds[a_index][max_moveIndexs[a_index]] if max_moveIndexs[a_index] < len(a_taskInds[a_index]) else task_num

        # perfom move
        old_t_index = allocation_structure[a_index]
        allocation_structure[a_index] = t_index
        coalition_structure[t_index].append(a_index)

        # update agents in the new coalition
        affected_a_indexes = []
        affected_t_indexes = []
        if t_index != task_num:
            affected_a_indexes.extend(coalition_structure[t_index])
            affected_t_indexes.append(t_index)

            # task_cons[i][t_index]
            for i in coalition_structure[t_index]:
                task_cons[i][t_index] = agent_contribution(agents, tasks, i, t_index, coalition_structure[t_index], constraints, gamma)
                cur_con[i] = task_cons[i][t_index]
        else:
            affected_a_indexes.append(a_index)
            task_cons[a_index][t_index] = 0
            cur_con[a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != task_num):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assign += 1
            coalition_structure[old_t_index].remove(a_index)
            affected_a_indexes.extend(coalition_structure[old_t_index])
            affected_t_indexes.append(old_t_index)
            for i in coalition_structure[old_t_index]:
                task_cons[i][old_t_index] = agent_contribution(agents, tasks, i, old_t_index, coalition_structure[old_t_index], constraints, gamma)
                cur_con[i] = task_cons[i][old_t_index]

        for i in affected_a_indexes:
            move_vals[i] = [
                task_cons[i][j] - cur_con[i]
                if j in a_taskInds[i] + [task_num]
                else -1000
                for j in range(0, task_num + 1)
            ]

        ## update other agents w.r.t the affected tasks
        for t_ind in affected_t_indexes:
            for i in range(0, agent_num):
                if (i not in coalition_structure[t_ind]) and (t_ind in a_taskInds[i]):
                    task_cons[i][t_ind] = agent_contribution(agents, tasks, i, t_ind, coalition_structure[t_ind], constraints, gamma)
                    move_vals[i][t_ind] = task_cons[i][t_ind] - cur_con[i]

        max_moveIndexs = [
            np.argmax([move_vals[i][j] for j in a_taskInds[i] + [task_num]])
            for i in range(0, agent_num)
        ]
        max_moveVals = [
            move_vals[i][a_taskInds[i][max_moveIndexs[i]]]
            if max_moveIndexs[i] < len(a_taskInds[i])
            else move_vals[i][task_num]
            for i in range(0, agent_num)
        ]

    return (
        coalition_structure,
        sys_rewards_tasks(tasks, agents, coalition_structure, gamma),
        iteration,
        re_assign,
    )


def eGreedyNE2(agents, tasks, constraints, agentIds, taskIds, coalition_structure=[], eps=0, gamma=1):
    """
    Solve the problem using GreedyNE, but only consider the agents and tasks in the given agentIds and taskIds
    """
    new_agents = [agents[i] for i in agentIds]
    new_tasks = [tasks[j] for j in taskIds]
    reverse_taskIds_map = {j: j1 for j1, j in enumerate(taskIds)}
    reverse_agentIds_map = {i: i1 for i1, i in enumerate(agentIds)}
    new_constraints = [[], []]
    for i in agentIds:
        new_constraints[0].append([reverse_taskIds_map[j] for j in constraints[0][i]])
    for j in taskIds:
        new_constraints[1].append([reverse_agentIds_map[i] for i in constraints[1][j]])
    coalition_structure, sys_reward, iteration_count, re_assign_count = eGreedy2(new_agents, new_tasks, new_constraints, eps, gamma, coalition_structure)
    return (
        [agentIds[i] for i in coalition_structure[j]]
        for j in range(0, len(taskIds))
    ), sys_reward, iteration_count, re_assign_count
    


def eGreedyDNF(agents, tasks, constraints, dnf_tree, eps=0, gamma=1, coalition_structure=[]):
    """
    Solve the problem using GreedyNE when the tasks are organized in a single OR-AND tree (Disjunctive Normal Form)
    """
    max_reward = 0
    for x, subtree in enumerate(dnf_tree):
        subtasksIds = [subtree] if isinstance(subtree, int) else subtree
        subagentIds = range(0, len(agents))
        coalition_structure, sys_reward, iteration_count, re_assign_count = eGreedyNE2(agents, tasks, constraints, subagentIds, subtasksIds, coalition_structure, eps, gamma)
        if sys_reward > max_reward:
            max_reward = sys_reward
            max_coalition_structure = coalition_structure
    return max_coalition_structure, max_reward, iteration_count, re_assign_count

In [ ]:


def resultCal(agents, tasks, constraints, r_msgs, q_msgs, iteration, iter_over, converge, gamma=1):
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)

    a_msg_sum = [
        {
            j: sum([r_msgs[j][i][0] for j in a_taskInds[i] if j != j]) + r_msgs[j][i][1]
            for j in a_taskInds[i]
        }
        for i in range(0, agent_num)
    ]

    alloc = [max(ams, key=ams.get) if ams != {} else task_num for ams in a_msg_sum]

    return (
        alloc,
        sys_reward_agents(agents, tasks, alloc, gamma),
        iteration,
        iter_over,
        converge,
    )



def convert_alloc_to_CS(tasks, allocation_structure):
    task_num = len(tasks)
    coalition_structure = [[] for i in range(0, task_num)]
    for i, j in enumerate(allocation_structure):
        if j < task_num:  # means allocated (!=task_num)
            coalition_structure[j].append(i)
    return coalition_structure





def FMS(agents, tasks, constraints, gamma, time_bound=500):
    '''
    Fast Max-Sum algorithm
    '''
    converge = False
    iter_over = False
    start_time = time.perf_counter()
    a_taskInds = constraints[0]
    t_agentInds = constraints[1]
    task_num = len(tasks)
    agent_num = len(agents)

    q_msgs = [{t_key: {} for t_key in a_taskInds[i]} for i in range(0, agent_num)]
    r_msgs = [
        {
            t_agentInds[j][i]: (
                {1: -100} if len(a_taskInds[t_agentInds[j][i]]) == 1
                else {key: -100 for key in [0, 1]}
            )
            for i in range(0, len(t_agentInds[j]))
        }
        for j in range(0, task_num)
    ]

    q_flags = [False for i in range(0, agent_num)]
    r_flags = [False for j in range(0, task_num)]

    iteration = 0
    while True:
        if time.perf_counter() - start_time >= time_bound:
            return resultCal(agents, tasks, constraints, r_msgs, q_msgs, iteration, iter_over, converge, gamma)
        iteration += 1

        if iteration > agent_num + task_num:
            iter_over = True
            return resultCal(agents, tasks, constraints, r_msgs, q_msgs, iteration, iter_over, converge, gamma)

        if all(q_flags) and all(r_flags):  # converge, msgs are all the same.
            converge = True
            # break
        for i in range(0, agent_num):
            linked_taskInds = a_taskInds[i]

            flag = True
            for t_key in linked_taskInds:
                ####### check time bound
                if time.perf_counter() - start_time >= time_bound:
                    return resultCal(agents, tasks, constraints, r_msgs, q_msgs, iteration, iter_over, converge, gamma)
                ####### check time bound
                msgs = {}

                if len(linked_taskInds) > 1:
                    msgs[1] = sum(
                        m[0]
                        for m in [
                            r_msgs[j][i] for j in linked_taskInds if j != t_key
                        ]
                    )
                    msg_0 = []
                    ts = list(linked_taskInds)
                    ts.remove(t_key)

                    for k in ts:
                        msg_0.append(
                            sum([m[0] for m in [r_msgs[j][i] for j in ts if j != k]])
                            + r_msgs[k][i][1]
                        )

                    msgs[0] = 0 if msg_0 == [] else max(msg_0)
                else:
                    msgs[1] = 0

                alphas = -sum(msgs.values()) / len(msgs.keys())

                msgs_regularised = {
                    d_key: msgs[d_key] + alphas for d_key in msgs.keys()
                }

                old_msg = q_msgs[i][t_key]
                if old_msg != {} and any(
                    abs(msgs_regularised[d_key] - old_msg[d_key]) > 10 ** (-5)
                    for d_key in old_msg.keys()
                ):
                    flag = False

                q_msgs[i][t_key] = msgs_regularised

            if flag:  # agent i sending the same info
                q_flags[i] = True

        if time.perf_counter() - start_time >= time_bound:
            break
        ###################### SAME thing, using comprehension
        #             msgs = {t_key:{d_key:sum([m[d_key] for m in [r_msgs[j][i] for j in linked_taskInds if j != t_key]])
        #                            for d_key in linked_taskInds}
        #                                 for t_key in linked_taskInds}
        #             alphas = {t_key:-sum(msgs[t_key].values())/len(msgs.keys())
        #                       for t_key in linked_taskInds}
        #             msgs_regularised = {t_key:{d_key:msgs[t_key][d_key] + alphas[t_key]
        #                            for d_key in linked_taskInds}
        #                                 for t_key in linked_taskInds}
        for j in range(0, task_num):
            linked_agentInds = t_agentInds[j]
            msg_con = [q_msgs[a][j] for a in linked_agentInds]

            com_dict = []
            com_rewards = []
            dom_com = [
                [0, 1] if len(a_taskInds[i]) > 1 else [1] for i in linked_agentInds
            ]

            for c in itertools.product(*dom_com):
                ####### check time bound
                if time.perf_counter() - start_time >= time_bound:
                    return resultCal(agents, tasks, constraints, r_msgs, q_msgs, iteration, iter_over, converge, gamma)
                ####### check time bound

                com_dict.append({linked_agentInds[i]: c[i] for i in range(0, len(c))})
                com_rewards.append(
                    task_reward(
                        tasks[j],
                        [agents[a_key] for a_key in com_dict[-1].keys() if com_dict[-1][a_key] == 1],
                        gamma,
                    )
                )

            flag = True
            for a_key in linked_agentInds:
                ####### check time bound
                if time.perf_counter() - start_time >= time_bound:
                    return resultCal(agents, tasks, constraints, r_msgs, q_msgs, iteration, iter_over, converge, gamma)
                ####### check time bound

                old_msg = r_msgs[j][a_key]
                q_table = []
                for c in range(0, len(com_dict)):
                    q_table.append(
                        sum([q_msgs[a][j][com_dict[c][a]] for a in linked_agentInds if a != a_key]) + com_rewards[c]
                    )

                r_msgs[j][a_key] = {
                    d_key: max(
                        [q_table[c] for c in range(0, len(com_dict)) if com_dict[c][a_key] == d_key]
                    )
                    for d_key in ([0, 1] if len(a_taskInds[a_key]) > 1 else [1])
                }

                if any( 
                    abs(r_msgs[j][a_key][d_key] - old_msg[d_key]) > 10 ** (-5)
                    for d_key in old_msg.keys()
                ):
                    flag = False

            if flag:  # task j sending the same info
                r_flags[j] = True

    return resultCal(agents, tasks, constraints, r_msgs, q_msgs, iteration, iter_over, converge, gamma)


def OPD(agents, tasks, constraints, gamma):
    edge_pruned = 0
    task_num = len(tasks)
    agent_num = len(agents)
    a_taskInds = [list(con) for con in constraints[0]]
    t_agentInds = [list(con) for con in constraints[1]]

    a_ubs = [[0 for j in a_taskInds[i]] for i in range(0, agent_num)]
    a_lbs = [[0 for j in a_taskInds[i]] for i in range(0, agent_num)]

    for j in range(0, task_num):
        linked_agentInds = t_agentInds[j]
        com_dict = []
        com_rewards = []
        for c in itertools.product(*[[0, 1] for i in linked_agentInds]):
            com_dict.append({linked_agentInds[i]: c[i] for i in range(0, len(c))})
            com_rewards.append(
                task_reward(
                    tasks[j],
                    [
                        agents[a_key]
                        for a_key in com_dict[-1].keys()
                        if com_dict[-1][a_key] == 1
                    ],
                    gamma,
                )
            )

        for i in t_agentInds[j]:
            t_ind = a_taskInds[i].index(j)
            a_ind = t_agentInds[j].index(i)
            reward_t = [
                com_rewards[c] for c in range(0, len(com_dict)) if com_dict[c][i] == 1
            ]
            reward_f = [
                com_rewards[c] for c in range(0, len(com_dict)) if com_dict[c][i] == 0
            ]
            cons_j = [reward_t[k] - reward_f[k] for k in range(0, len(reward_t))]
            a_lbs[i][t_ind] = min(cons_j)
            a_ubs[i][t_ind] = max(cons_j)

    for i in range(0, agent_num):
        t_flag = [True for j in a_taskInds[i]]
        for t_ind in range(0, len(a_taskInds[i])):
            for t2_ind in range(0, len(a_taskInds[i])):
                if t_ind != t2_ind and a_ubs[i][t_ind] < a_lbs[i][t2_ind]:
                    t_flag[t_ind] = False
                    edge_pruned += 1
                    break

        for t_ind in range(0, len(t_flag)):
            if not t_flag[t_ind]:
                t_agentInds[a_taskInds[i][t_ind]].remove(i)

        new_a_taskInds = [
            a_taskInds[i][t_ind]
            for t_ind in range(0, len(a_taskInds[i]))
            if t_flag[t_ind]
        ]
        a_taskInds[i] = new_a_taskInds

    return a_taskInds, t_agentInds, edge_pruned


def FMSNormalised(agents,tasks,constraints,gamma,time_bound = 500):
    converge = False
    iter_over = False
    start_time = time.perf_counter()
    a_taskInds = constraints[0]
    t_agentInds = constraints[1]
    task_num = len(tasks)
    agent_num =len(agents)
    
    q_msgs = [{t_key:{} for t_key in a_taskInds[i]} for i in range(0,agent_num)]
    r_msgs = [{t_agentInds[j][i]:({1:-100} if len(a_taskInds[t_agentInds[j][i]])==1 else {key:-100 for key in [0,1]})
               for i in range(0,len(t_agentInds[j]))}
              for j in range(0,task_num)]

    
    q_flags = [False for i in range(0,agent_num)]
    r_flags = [False for j in range(0,task_num)] 
    
    iteration = 0
    while True:
        if time.perf_counter() - start_time >= time_bound:
            return resultCal(agents,tasks,constraints,r_msgs,q_msgs,iteration,iter_over,converge,gamma)
        iteration += 1
        
        if iteration > agent_num +task_num:
            iter_over = True
            return resultCal(agents,tasks,constraints,r_msgs,q_msgs,iteration,iter_over,converge,gamma)
        
        if all(q_flags) and all(r_flags): #converge, msgs are all the same.
            converge = True
#             break
        for i in range(0,agent_num):
            linked_taskInds = a_taskInds[i]
            
            flag = True
            for t_key in linked_taskInds:
                
                ####### check time bound
                if time.perf_counter() - start_time >= time_bound:
                    return resultCal(agents,tasks,constraints,r_msgs,q_msgs,iteration,iter_over,converge,gamma)
                ####### check time bound
                msgs = {}
                
                
                if len(linked_taskInds)>1:
                    msgs[1] = sum([m[0] for m in [r_msgs[j][i] for j in linked_taskInds if j != t_key]])
                    msg_0 = []
                    ts = list(linked_taskInds)
                    ts.remove(t_key)
                
                    for k in ts:
                        msg_0.append(sum([m[0] for m in [r_msgs[j][i] for j in ts if j != k]]) 
                                     + r_msgs[k][i][1])
                    
                    msgs[0]= (0 if msg_0 == [] else max(msg_0))
                else:
                    msgs[1] = 0
            
                alphas = -sum(msgs.values())/len(msgs.keys())
            
                msgs_regularised = {d_key:msgs[d_key] + alphas for d_key in msgs.keys()} 
                
                
                old_msg = q_msgs[i][t_key]
                if old_msg!={} and any([abs(msgs_regularised[d_key] - old_msg[d_key]) > 10**(-5) 
                                        for d_key in old_msg.keys()]):
                    flag = False
                
                q_msgs[i][t_key] = msgs_regularised         
            
            if flag: # agent i sending the same info
                q_flags[i] = True
        
        if time.perf_counter() - start_time >= time_bound:
            break
###################### SAME thing, using comprehension                
#             msgs = {t_key:{d_key:sum([m[d_key] for m in [r_msgs[j][i] for j in linked_taskInds if j != t_key]])
#                            for d_key in linked_taskInds} 
#                                 for t_key in linked_taskInds}            
#             alphas = {t_key:-sum(msgs[t_key].values())/len(msgs.keys()) 
#                       for t_key in linked_taskInds}            
#             msgs_regularised = {t_key:{d_key:msgs[t_key][d_key] + alphas[t_key]
#                            for d_key in linked_taskInds} 
#                                 for t_key in linked_taskInds}        
        for j in range(0, task_num):
            linked_agentInds = t_agentInds[j]            
            msg_con = [q_msgs[a][j] for a in linked_agentInds]
            
            com_dict = []
            com_rewards = []
            dom_com = [[0,1] if len(a_taskInds[i]) > 1 else [1] for i in linked_agentInds]

            for c in itertools.product(*dom_com):
                ####### check time bound
                if time.perf_counter() - start_time >= time_bound:
                    return resultCal(agents,tasks,constraints,r_msgs,q_msgs,iteration,iter_over,converge,gamma)
                ####### check time bound
                
                com_dict.append({linked_agentInds[i]:c[i] for i in range(0,len(c))})
                com_rewards.append(
                    task_reward(tasks[j],[agents[a_key] for a_key in com_dict[-1].keys() if com_dict[-1][a_key] == 1],gamma)
                )
                
            
            flag = True
            for a_key in linked_agentInds:
                
                ####### check time bound
                if time.perf_counter() - start_time >= time_bound:
                    return resultCal(agents,tasks,constraints,r_msgs,q_msgs,iteration,iter_over,converge,gamma)
                ####### check time bound
                
                old_msg = r_msgs[j][a_key]
                q_table = []
                for c in range(0,len(com_dict)):
                    q_table.append(sum([q_msgs[a][j][com_dict[c][a]] for a in linked_agentInds if a != a_key]) 
                                          + com_rewards[c])
                
                r_msgs[j][a_key] = {d_key:max([q_table[c] for c in range(0,len(com_dict)) if com_dict[c][a_key] == d_key]) 
                                      for d_key in ([0,1] if len(a_taskInds[a_key])>1 else [1])}
                

                if any([abs(r_msgs[j][a_key][d_key] - old_msg[d_key]) > 10**(-5) for d_key in old_msg.keys()]):
                    flag = False
                    

            
            if flag: # task j sending the same info
                r_flags[j] = True
                
    return resultCal(agents,tasks,constraints,r_msgs,q_msgs,iteration,iter_over,converge,gamma)
           


def random_solution_heterogeneous(agents, tasks, constraints, gamma=1):
    '''
    Randomly allocate tasks to agents
    '''
    task_num = len(tasks)
    agent_num = len(agents)
    a_taskInds = constraints[0]
    alloc = [np.random.choice(a_taskInds[i] + [task_num]) for i in range(0, agent_num)]
    return alloc, sys_reward_agents(agents, tasks, alloc, gamma)



In [ ]:
def upperBound(capabilities, tasks, agents):
    """
    Calculate the upper bound of the system reward, where the system consists of tasks and agents with constraints.

    This mathematical upper bound is calculated by sorting the agents based on their contribution values for each capability, in descending order, then count `m`, the number of tasks that require each capability, and sum up the contribution values of the top `m` agents for each capability.
    
    :param: `capabilities`: the list of capabilities
    :param: `tasks`: the list of tasks
    :param: `agents`: the list of agents
    :return: the upper bound of the system reward
    """
    cap_ranked = [sorted([a[c] for a in agents], reverse=True) for c in capabilities] # Time complexity: O(len(capabilities) * log(len(capabilities)) * len(agents))
    cap_req_all = list(itertools.chain(*tasks)) # Time complexity: O(size of tasks capabilities combined), around O(len(tasks) * len(capabilities))
    cap_req_num = [cap_req_all.count(c) for c in capabilities] # Time complexity: O(len(cap_req_all) * len(capabilities)). However, can be optimized to O(len(cap_req_all)).
    return sum([sum(cap_ranked[c][:cap_req_num[c]]) for c in capabilities]) # Time complexity: O(len(cap_req_all))
    # Evaluated time complexity: max(O(len(capabilities) * log(len(capabilities)) * len(agents)), O(len(tasks) * len(capabilities)))


def upperBound_ver2(capabilities, tasks, agents, constraints):
    """
    Calculate the upper bound of the system reward, where the system consists of tasks and agents with constraints.

    This upper bound is calculated by sorting the agents based on their contribution values for each capability, in descending order, then iteratively allocate the top agents to the tasks that require that capability.

    This allows for a more precise upper bound than upperBound, since it takes into account the `constraints`: the top agents might only be able to work on the same limited tasks.

    :param: `capabilities`: the list of capabilities
    :param: `tasks`: the list of tasks
    :param: `agents`: the list of agents
    :param: `constraints`: the list of constraints
    :return: the upper bound of the system reward
    """
    agent_num = len(agents)
    task_num = len(tasks)
    a_taskInds = constraints[0]
    cap_req_all = list(itertools.chain(*tasks))
    cap_req_num = [cap_req_all.count(c) for c in capabilities]

    sys_rewards = 0
    for c in capabilities:
        
        a_cap_vals = [agent[c] for agent in agents]

        # the list of tasks that each agent has the capability to perform and that require the capability c
        a_cap_tasks = [[j for j in a_taskInd if j != task_num and c in tasks[j]] for a_taskInd in a_taskInds] 

        # sort the agents based on their contribution values for the capability c, in descending order
        cap_rank_pos = np.argsort(a_cap_vals)[::-1]

        a_cap_vals_ordered = [0 for _ in range(0, agent_num)]
        a_cap_tasks_ordered = [[] for _ in range(0, agent_num)]
        for p, pos in enumerate(cap_rank_pos):
            a_cap_vals_ordered[p] = a_cap_vals[pos]
            a_cap_tasks_ordered[p] = a_cap_tasks[pos]

        cap_rewards = a_cap_vals_ordered[0]
        cap_tasks = set(a_cap_tasks_ordered[0])
        a_cap_num = 1
        for a_iter in range(1, agent_num):
            cap_tasks = cap_tasks.union(set(a_cap_tasks_ordered[a_iter]))
            if len(cap_tasks) > a_cap_num:
                cap_rewards += a_cap_vals_ordered[a_iter]
                a_cap_num += 1
            # break if they got enough agents to contribute the number of required cap c
            if (a_cap_num >= cap_req_num[c]):  
                break
        sys_rewards += cap_rewards
    return sys_rewards



In [ ]:


class NodeType(Enum):
    AND = "AND"
    OR = "OR"
    LEAF = "LEAF"
    DUMMY = "DUMMY"

def reverse_node_type(node_type):
    if node_type == NodeType.AND:
        return NodeType.OR
    elif node_type == NodeType.OR:
        return NodeType.AND
    else:
        return node_type
    

def is_not_leaf(node_type):
    return node_type == NodeType.AND or node_type == NodeType.OR


@dataclass
class Node:
    node_id: int
    node_type: NodeType = None
    parent_id: int = None
    depth: int = None
    children_ids: list[int] = None

def gen_random_partition(array, flatten: bool = True):
    """
    Generate a random partition of the given array.
    """
    if len(array) <= 2:
        return array
    partition = []
    for element in array:
        random_index = np.random.randint(0, len(partition) + 1)
        if random_index == len(partition):
            partition.append([element])
        else:
            partition[random_index].append(element)

    if flatten:
        for i, subset in enumerate(partition):
            if len(subset) == 1:
                partition[i] = partition[i][0]

    if len(partition) == 1:
        partition = partition[0]

    return partition


def gen_tree_info(task_num, max_depth=None):
    """
    Generate a random AND/OR tree with `task_num` tasks.
    Returns a list of lists or integers, where each element/subelement is a node.
    Each leaf node is an integer from 0 to `task_num-1`, and each non-leaf node is a list.
    """
    if max_depth is None:
        max_depth = np.random.randint(1, task_num // 2)
    # Fix max_depth to be at least 1 and at most task_num
    max_depth = max(1, min(max_depth, task_num))

    # We reserve task_num for the id of the dummy task coalition.
    tree_info = [
        Node(
            node_id=i,
            node_type=NodeType.LEAF,
            parent_id=None,
            depth=0,
            children_ids=[]
        ) for i in range(task_num)
    ] + [
        Node(
            node_id=task_num,
            node_type=NodeType.DUMMY,
            parent_id=None,
            depth=None,
            children_ids=[]
        )
    ]
    
    global_node_id = task_num
    node_ids_list = list(range(task_num))
    non_leaf_node_type = np.random.choice([NodeType.AND, NodeType.OR])
    depth = 0
    for _ in range(max_depth - 1):
        partition = gen_random_partition(node_ids_list, flatten=True)
        node_ids_list = []
        depth += 1
        non_leaf_node_type = reverse_node_type(non_leaf_node_type)
        for subset in partition:
            if isinstance(subset, list):
                global_node_id += 1
                tree_info.append(Node(
                    node_id=global_node_id,
                    node_type=np.random.choice([NodeType.AND, NodeType.OR]),
                    parent_id=None,
                    depth=depth,
                    children_ids=subset
                ))
                for leaf in subset:
                    tree_info[leaf].parent_id = global_node_id
                node_ids_list.append(global_node_id)
            else:
                tree_info[subset].parent_id = global_node_id
                tree_info[subset].depth = depth
                node_ids_list.append(subset)
    
    # Root node
    depth += 1
    global_node_id += 1
    tree_info.append(Node(
        node_id=global_node_id,
        node_type=np.random.choice([NodeType.AND, NodeType.OR]),
        parent_id=None,
        depth=depth,
        children_ids=node_ids_list
    ))
    for subset in node_ids_list:
        tree_info[subset].parent_id = global_node_id

    # Fix depth values, since we have been using the reversed depth
    for node in tree_info:
        if node.depth is not None:
            node.depth = depth - node.depth

    return tree_info


def traverse_tree_info(tree_info : list[Node], order='dfs', root_node_index=-1):
    """
    Traverse tree using depth-first search.
    
    Returns a generator that yields the node id, node type, parent node id, and depth of each node.
    
    To get the node id, node type, parent node id, and depth of the root node, use `next(generator)`.
    
    To iterate through the generator, use `for node_id, node_type, parent, depth in generator:`.

    For each `num_tasks`, the total number of non-leaf nodes is at most `num_tasks - 1`.
    
    For each node, `node_id` is iterated in ascending order starting from `num_tasks + 1` (unless it is a leaf, then node_id is the node). We reserve `num_tasks` for the id of the dummy task coalition.

    For each node, `node_type` is either 'AND', 'OR', or 'LEAF'.
    """
    frontier_pop_index = 0 if order.lower() == 'bfs' else -1
    frontier = [tree_info[root_node_index]]
    while len(frontier) > 0:
        node = frontier.pop(frontier_pop_index)
        yield node
        if node.children_ids is not None:
            for child_id in node.children_ids:
                frontier.append(tree_info[child_id])


def convert_tree_to_list(tree_info : list[Node]):
    """
    Convert tree_info to tree_list, a list of lists or integers, where each element/subelement is a node.
    """
    def _helper(node : Node):
        if node.children_ids is None or node.children_ids == []:
            return node.node_id
        return [_helper(tree_info[i]) for i in node.children_ids]
    return _helper(tree_info[-1])


def normal_form_tree_info(tree_info : list[Node], form : str = 'DNF'):
    """
    Convert a tree to a normal form. (CNF or DNF)
    Worst case Complexity: At least O(e^(n/e)) where n is the number of leafs in the tree.
    """

    def _normalized(node : Node):
        print(node.node_id)
        if not is_not_leaf(node.node_type):
            return node.node_id
        # Flatten the tree
        new_children_ids = []
        for child_id in node.children_ids:
            if tree_info[child_id].node_type == node.node_type:
                new_children_ids.extend(tree_info[child_id].children_ids)
            else:
                new_children_ids.append(child_id)

        new_tree = [_normalized(tree_info[i]) for i in new_children_ids]

        list_clauses = [clause for clause in new_tree if isinstance(clause, list)]
        literal_clauses = [clause for clause in new_tree if not isinstance(clause, list)]
        target_node_type = NodeType.AND if form.upper() == 'CNF' else NodeType.OR
        
        if node.node_type == target_node_type:
            return literal_clauses + [leaf for subtree in list_clauses for leaf in subtree]
        else:
            new_normal_tree = []
            for combination in itertools.product(*list_clauses):
                new_combination = []
                # Flatten the combination
                for item in combination:
                    if isinstance(item, list):
                        new_combination.extend(item)
                    else:
                        new_combination.append(item)
                # Add the literal clauses
                new_combination.extend(literal_clauses)
                # Add the new combination to the new tree
                new_normal_tree.append(new_combination)
            return new_normal_tree
        
    return _normalized(tree_info[-1])


def get_leafs_for_each_node(tree_info : list[Node], root_node_index=-1):
    leafs_list = [[] for _ in range(len(tree_info))]
    leaf_nodes = [node for node in tree_info if node.node_type == NodeType.LEAF]
    for leaf_node in leaf_nodes:
        leafs_list[leaf_node.node_id].append(leaf_node.node_id)
        parent_id = leaf_node.parent_id
        while parent_id is not None:
            leafs_list[parent_id].append(leaf_node.node_id)
            parent_id = tree_info[parent_id].parent_id
    return leafs_list


def get_nodes_constraints(tree_info : list[Node], constraints, root_node_index=-1):
    """
    Get the constraints for each node in the tree.
    """
    nodes_agents = [[] for _ in range(len(tree_info))]
    leafs_list = get_leafs_for_each_node(tree_info)
    for node in tree_info:
        if node.node_type == NodeType.LEAF:
            nodes_agents[node.node_id] = constraints[1][node.node_id]
        else:
            nodes_agents[node.node_id] = list(set(itertools.chain(
                *[constraints[1][leaf] for leaf in leafs_list[node.node_id]]
            ))) # Concat lists and remove duplicates

    a_nodes = [[] for a in constraints[0]]
    for node_id, node_As in enumerate(nodes_agents):
        for a in node_As:
            a_nodes[a].append(node_id)

    return a_nodes, nodes_agents


def get_children_info(parent_info : dict[int, int]):
    """
    Get the children info for each node in the tree.
    """
    children_info = { parent_id: [] for parent_id in parent_info.values() }
    for node_id, parent_id in parent_info.items():
        children_info[parent_id].append(node_id)
    return children_info


def get_leafs_info(parent_info : dict[int, int], leaf_ids : list[int]):
    """
    Get the leaf info for each node in the tree.
    """
    leafs_info = { node_id: [] for node_id in parent_info }
    for leaf_id in leaf_ids:
        leafs_info[leaf_id].append(leaf_id)
        node_id = parent_info[leaf_id]
        while node_id is not None:
            leafs_info[node_id].append(leaf_id)
            node_id = parent_info[node_id]
    return leafs_info


def get_parents_sequence(tree_info : list[Node]):
    """
    Get the parents sequence for each leaf in the tree.
    """
    leaf_nodes = [node for node in tree_info if node.node_type == NodeType.LEAF]
    parents_sequence = { leaf_node.node_id: [] for leaf_node in leaf_nodes }
    for leaf_node in leaf_nodes:
        parents_sequence[leaf_node.node_id].append(leaf_node.node_id)
        parent_id = leaf_node.parent_id
        while parent_id is not None:
            parents_sequence[leaf_node.node_id].append(parent_id)
            parent_id = tree_info[parent_id].parent_id
    return parents_sequence

In [ ]:
def calc_upper_bound_vectors_tree(tree_info: list[Node], capabilities, tasks, root_node_index=-1):
    nodes_upperBound_capVector = [np.zeros(len(capabilities)) for _ in range(0, len(tree_info))]
    tasks_capVector = [np.array([1 if c in tasks[j] else 0 for c in capabilities]) for j in range(0, len(tasks))]

    def _calc_upper_bound_vector_node(node_index : int):
        node = tree_info[node_index]
        if node.node_type == NodeType.LEAF:
            nodes_upperBound_capVector[node_index] = tasks_capVector[node.node_id]
            return nodes_upperBound_capVector[node_index]
        if node.node_type == NodeType.OR:
            for child_index in node.children_ids:
                _calc_upper_bound_vector_node(child_index)
            nodes_upperBound_capVector[node_index] = np.max([nodes_upperBound_capVector[child_index] for child_index in node.children_ids], axis=0)
            return nodes_upperBound_capVector[node_index]
        if node.node_type == NodeType.AND:
            for child_index in node.children_ids:
                _calc_upper_bound_vector_node(child_index)
            nodes_upperBound_capVector[node_index] = np.sum([nodes_upperBound_capVector[child_index] for child_index in node.children_ids], axis=0)
            return nodes_upperBound_capVector[node_index]
    
    _calc_upper_bound_vector_node(root_node_index)
    return nodes_upperBound_capVector
    

def upperBoundTree_root(tree_info: list[Node], capabilities, tasks, agents, root_node_index=-1):
    """
    Calculate the upper bound of the system reward, i.e. at the root of the AND-OR goal tree.
    """
    nodes_upperBound_capVector = calc_upper_bound_vectors_tree(tree_info, capabilities, tasks, root_node_index)
    caps_ranked = [sorted([a[c] for a in agents], reverse=True) for c in capabilities]
    
    return sum([sum(caps_ranked[c][:int(nodes_upperBound_capVector[root_node_index][c])]) for c in capabilities])


def upperBoundTree_allNodes_v1(tree_info: list[Node], capabilities, tasks, agents, nodes_constraints, root_node_index=-1):
    """
    Calculate the upper bound of the reward (utility) at each node of the AND-OR goal tree.
    """
    nodes_upperBound_capVector = calc_upper_bound_vectors_tree(tree_info, capabilities, tasks, root_node_index)
    
    nodes_agents = nodes_constraints[1]
    
    nodes_caps_ranked = [[sorted([agents[i][c] for i in nodes_agents[node_index]], reverse=True) for c in capabilities] for node_index in range(0, len(tree_info))]
    
    return [sum([sum(nodes_caps_ranked[node_index][c][:int(nodes_upperBound_capVector[node_index][c])]) for c in capabilities]) for node_index in range(0, len(tree_info))]



def upperBoundTree_allNodes_v2(tree_info: list[Node], capabilities, tasks, agents, nodes_constraints, root_node_index=-1):
    """
    Calculate the upper bound of the reward (utility) at each node of the AND-OR goal tree.
    """
    nodes_upperBound_capVector = calc_upper_bound_vectors_tree(tree_info, capabilities, tasks, root_node_index)
    
    nodes_agents = nodes_constraints[1]
    
    nodes_caps_ranked = [[sorted([agents[i][c] for i in nodes_agents[node_index]], reverse=True) for c in capabilities] for node_index in range(0, len(tree_info))]
    
    nodes_upper_bound = [sum([sum(nodes_caps_ranked[node_index][c][:int(nodes_upperBound_capVector[node_index][c])]) for c in capabilities]) for node_index in range(0, len(tree_info))]

    nodes_upper_bound_min = [0 for node_index in range(0, len(tree_info))]

    def _min_upper_bound(node_id : int):
        node = tree_info[node_id]
        if node.node_type == NodeType.LEAF:
            nodes_upper_bound_min[node_id] = nodes_upper_bound[node_id]
            return nodes_upper_bound_min[node_id]
        if node.node_type == NodeType.OR:
            nodes_upper_bound_min[node_id] = max(_min_upper_bound(child_index) for child_index in node.children_ids)
            nodes_upper_bound_min[node_id] = min(nodes_upper_bound[node_id], nodes_upper_bound_min[node_id])
            return nodes_upper_bound_min[node_id]
        if node.node_type == NodeType.AND:
            nodes_upper_bound_min[node_id] = sum(_min_upper_bound(child_index) for child_index in node.children_ids)
            nodes_upper_bound_min[node_id] = min(nodes_upper_bound[node_id], nodes_upper_bound_min[node_id])
            return nodes_upper_bound_min[node_id]
        if node.node_type == NodeType.DUMMY:
            nodes_upper_bound_min[node_id] = 0
            return nodes_upper_bound_min[node_id]
        else:
            raise Exception("Unknown node type")
        
    _min_upper_bound(root_node_index)

    return nodes_upper_bound_min

In [ ]:
def agent_contribution(agents, tasks, query_agentIndex, query_taskIndex, coalition, constraints, gamma=1):
    """
    Return contribution of agent i to task j in coalition C_j
    
    = U_i(C_j, j) - U_i(C_j \ {i}, j) if i in C_j

    = U_i(C_j U {i}, j) - U_i(S, j) if i not in C_j
    """
    a_taskInds = constraints[0]
    if query_taskIndex == len(tasks):
        return 0
    if query_taskIndex not in a_taskInds[query_agentIndex]:
        return 0
    cur_reward = task_reward(tasks[query_taskIndex], [agents[i] for i in coalition], gamma)
    if query_agentIndex in coalition:
        return cur_reward - task_reward(tasks[query_taskIndex], [agents[i] for i in coalition if i != query_agentIndex], gamma)
    else:
        return task_reward(tasks[query_taskIndex], [agents[i] for i in coalition] + [agents[query_agentIndex]], gamma) - cur_reward


def calc_temp_node_values(query_a_index, tasks, tree_info, allocation_structure, cur_con, realtime_node_values, root_node_index):
    """
    Calculate temp_node_values[i], for when agent i is removed from the system
    """
    temp_node_values_i = realtime_node_values.copy()
    a_current_t_id = allocation_structure[query_a_index]

    sys_lost_value = cur_con[query_a_index]
    temp_node_values_i[a_current_t_id] -= sys_lost_value
    
    parent_id = tree_info[a_current_t_id].parent_id
    while parent_id is not None and parent_id != len(tasks) and sys_lost_value > 0:
        
        if(tree_info[parent_id].node_type == NodeType.AND):
            temp_node_values_i[parent_id] -= sys_lost_value
        
        elif(tree_info[parent_id].node_type == NodeType.OR):
            new_parent_value = max([temp_node_values_i[j] for j in tree_info[parent_id].children_ids])
            sys_lost_value = temp_node_values_i[parent_id] - new_parent_value
            temp_node_values_i[parent_id] = new_parent_value

        if parent_id == root_node_index:
            break
        parent_id = tree_info[parent_id].parent_id

    return temp_node_values_i



def calc_nodes_alt_values(agent_num, task_num, tasks, tree_info, a_taskInds, allocation_structure, task_cons, realtime_node_values, temp_node_values):
    """
    Calculate the movement values for all agents
    """
    nodes_alt_values = [[[] for j in range(0, task_num)] for i in range(0, agent_num)]

    for i in range(0, agent_num):
        
        # Calculate the best move values for each agent
        for j in a_taskInds[i]:

            if j == allocation_structure[i]:
                nodes_alt_values[i][j] = realtime_node_values.copy()
                continue

            nodes_alt_values[i][j] = temp_node_values[i].copy()
        
            sys_added_value = task_cons[i][j]
            nodes_alt_values[i][j][j] = temp_node_values[i][j] + sys_added_value
            node_id = j
            parent_id = tree_info[j].parent_id

            # Backtrack to the root node
            while parent_id is not None and parent_id != len(tasks) and sys_added_value > 0:

                # Break conditions: 
                # parent_id is invalid (None) (i.e, node_id is root node) or parent_id is the dummy node
                # sys_added_value <= 0

                # max_move_value only increases, and thus (max_move_value - move_vals_exit) always >= 0
                # meanwhile, sys_added_value only decreases

                parent_val = nodes_alt_values[i][j][parent_id]
                
                if(tree_info[parent_id].node_type == NodeType.AND):
                    nodes_alt_values[i][j][parent_id] += sys_added_value
                
                elif(tree_info[parent_id].node_type == NodeType.OR):
                    if (nodes_alt_values[i][j][parent_id] < nodes_alt_values[i][j][node_id]):
                        sys_added_value = nodes_alt_values[i][j][node_id] - nodes_alt_values[i][j][parent_id]
                        nodes_alt_values[i][j][parent_id] += sys_added_value
                    else:
                        sys_added_value = 0
                        break

                node_id = parent_id
                parent_id = tree_info[parent_id].parent_id
            
        
    return nodes_alt_values


def calc_max_move_value(agent_num, task_num, tasks, tree_info, a_taskInds, allocation_structure, cur_con, task_cons, realtime_node_values, temp_node_values, root_node_index=-1):
    """
    Calculate the best move values for agent query_a_index
    """
    # Max move values for each agent. Move value of an agent to new coalition j is the difference in the system value when the agent is moved from its current coalition to j.
    max_moves = [(float("-inf"), float("-inf"), task_num) for i1 in range(0, agent_num)]

    for i in range(0, agent_num):
        # Movement value for moving agent from current coalition to dummy coalition (removing the agent from the system):
        sys_exit_value = temp_node_values[i][root_node_index] - realtime_node_values[root_node_index]

        # Initialize the max move value
        max_moves[i] = (sys_exit_value, 0, task_num)
        
        # Calculate the best move values for each agent
        for j in a_taskInds[i]:

            if j == allocation_structure[i]:
                continue
        
            sys_added_value = task_cons[i][j]
            node_val = temp_node_values[i][j] + sys_added_value
            parent_id = tree_info[j].parent_id

            while parent_id is not None and parent_id != len(tasks) and (sys_added_value + sys_exit_value) >= max_moves[i][0] and sys_added_value > 0:

                # Break conditions: 
                # parent_id is invalid (None) (i.e, node_id is root node) or parent_id is the dummy node
                # sys_added_value <= 0
                # (sys_added_value + sys_exit_value) < best_move_value

                # max_move_value only increases, and thus (max_move_value - move_vals_exit) always >= 0
                # meanwhile, sys_added_value only decreases

                parent_val = temp_node_values[i][parent_id]
                
                if(tree_info[parent_id].node_type == NodeType.AND):
                    parent_val += sys_added_value
                
                elif(tree_info[parent_id].node_type == NodeType.OR):
                    if (parent_val < node_val):
                        sys_added_value = node_val - parent_val
                        parent_val = node_val
                    else:
                        sys_added_value = 0
                        break

                node_val = parent_val
                if parent_id == root_node_index:
                    break
                parent_id = tree_info[parent_id].parent_id
            
            move_val_j = sys_exit_value + sys_added_value
            
            if move_val_j > max_moves[i][0]:
                max_moves[i] = (move_val_j, task_cons[i][j] - cur_con[i], j)
            elif(move_val_j == max_moves[i][0]):
                # Tie breaking: choose the one with higher contribution
                if task_cons[i][j] > task_cons[i][max_moves[i][2]]:
                    max_moves[i] = (move_val_j, task_cons[i][j] - cur_con[i], j)
                elif task_cons[i][j] == task_cons[i][max_moves[i][2]]:
                    # Tie breaking: choose the one that moves out of the dummy coalition
                    if max_moves[i][2] == task_num or j < max_moves[i][2]:
                        max_moves[i] = (move_val_j, task_cons[i][j] - cur_con[i], j)

    return max_moves
    

def calc_best_move(agent_num, task_num, tasks, tree_info, a_taskInds, allocation_structure, cur_con, task_cons, realtime_node_values, temp_node_values, root_node_index=-1):
    """
    Calculate the best move values among all agents
    """
    best_move = (float("-inf"), float("-inf"), allocation_structure[0], 0)
    for i in range(0, agent_num):
        # Movement value for moving agent from current coalition to dummy coalition (removing the agent from the system):
        sys_exit_value = temp_node_values[i][root_node_index] - realtime_node_values[root_node_index]
        
        # Calculate the best move values for each agent
        for j in a_taskInds[i]:

            if j == allocation_structure[i]:
                continue
        
            sys_added_value = task_cons[i][j]
            node_val = temp_node_values[i][j] + sys_added_value
            parent_id = tree_info[j].parent_id

            # Backtrack to the root node
            while parent_id is not None and parent_id != len(tasks) and (sys_added_value + sys_exit_value) >= best_move[0] and sys_added_value > 0:

                # Break conditions: 
                # parent_id is invalid (None) (i.e, node_id is root node) or parent_id is the dummy node
                # sys_added_value <= 0
                # (sys_added_value + sys_exit_value) < best_move_value

                # max_move_value only increases, and thus (max_move_value - move_vals_exit) always >= 0
                # meanwhile, sys_added_value only decreases

                parent_val = temp_node_values[i][parent_id]
                
                if(tree_info[parent_id].node_type == NodeType.AND):
                    parent_val += sys_added_value
                
                elif(tree_info[parent_id].node_type == NodeType.OR):
                    if (parent_val < node_val):
                        sys_added_value = node_val - parent_val
                        parent_val = node_val
                    else:
                        sys_added_value = 0
                        break

                node_val = parent_val
                if parent_id == root_node_index:
                    break
                parent_id = tree_info[parent_id].parent_id
            
            # Compare the calculated system move value with the best move value

            sys_move_val_i_j = sys_exit_value + sys_added_value
            task_move_val_i_j = task_cons[i][j] - cur_con[i]

            if(sys_move_val_i_j < best_move[0]):
                continue

            if sys_move_val_i_j > best_move[0]:
                best_move = (sys_move_val_i_j, task_move_val_i_j, j, i)
                continue       
            
            # else: sys_move_val_i_j == best_move[0]:
            
            # Tie breaking: choose the one with higher contribution to a single task
            if task_move_val_i_j > best_move[1]:
                best_move = (sys_move_val_i_j, task_move_val_i_j, j, i)
                continue
            
            # Tie breaking: choose the one that moves out of the dummy coalition
            if task_move_val_i_j == best_move[1] and allocation_structure[i] == task_num:
                best_move = (sys_move_val_i_j, task_move_val_i_j, j, i)
                continue
        
    return best_move


def greedyNETree(agents, tasks, constraints, tree_info : list[Node], root_node_index=-1, agentIds=None, eps=0, gamma=1, coalition_structure=[], greedy_level=2, cur_con=None, task_cons=None, realtime_node_values=None, temp_node_values=None):
    """
    GreedyNE algorithm for solving the problem when the tasks are organized in strictly alternating AND-OR tree (i.e. each OR node has only AND children, and each AND node has only OR children)
    """    
    task_num = len(tasks)
    agent_num = len(agents)
    if agentIds is None:
        agentIds = list(range(0, agent_num))

    if root_node_index < 0:
        root_node_index = len(tree_info) + root_node_index
    
    nodes = list(traverse_tree_info(tree_info, order='dfs', root_node_index=root_node_index)) + [tree_info[task_num]]
    leaf_nodes = [node.node_id for node in nodes if node.node_type == NodeType.LEAF]
    a_taskInds = [[j for j in constraints[0][i] if j in leaf_nodes] for i in range(0, len(agents))]
    
    # each indicate the current task that agent i is allocated to, if = N, means not allocated
    allocation_structure = [task_num for i in range(0, agent_num)]

    # Initialize the coalition structure and contribution values of each agent to its current task
    if coalition_structure == None or coalition_structure == []:
        coalition_structure = [[] for j in range(0, task_num)] + [list(range(0, agent_num))]  # current coalition structure, the last one is dummy coalition
        if cur_con is None or cur_con == []:
            cur_con = [0 for i in range(0, agent_num)]
    else:
        for j in range(0, task_num + 1):
            for i in coalition_structure[j]:
                allocation_structure[i] = j
        # Contribution values of each agent to its current task
        if cur_con is None or cur_con == []:
            cur_con = [
                agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
                for i, j in enumerate(allocation_structure)
            ]

    # Contribution values of each agent to each task
    if task_cons is None or task_cons == []:
        task_cons = [
            [
                agent_contribution(agents, tasks, i, j, coalition_structure[j], constraints, gamma)
                if j in a_taskInds[i]
                else float("-inf")
                for j in range(0, task_num)
            ] + [0]
            for i in range(0, agent_num)
        ]


    # Node values for the current coalition structure
    if realtime_node_values is None or realtime_node_values == []:
        realtime_node_values = [0 for node in range(0, len(tree_info))]
    
    # temp_node_values is used to store the alternative node values when the agents are removed from the system (i.e., moved to the dummy coalition)
    if temp_node_values is None or temp_node_values == []:
        temp_node_values = [[0 for node in range(0, len(tree_info))] for i in range(0, agent_num)]

        for i in agentIds:
            temp_node_values[i] = calc_temp_node_values(i, tasks, tree_info, allocation_structure, cur_con, realtime_node_values, root_node_index=root_node_index)

    match (greedy_level):
        case 2:
            best_move_NE = calc_best_move(agent_num, task_num, tasks, tree_info, a_taskInds, allocation_structure, cur_con, task_cons, realtime_node_values, temp_node_values, root_node_index=root_node_index)
        case 1:
            max_moves = calc_max_move_value(agent_num, task_num, tasks, tree_info, a_taskInds, allocation_structure, cur_con, task_cons, realtime_node_values, temp_node_values, root_node_index=root_node_index)
        case _:
            node_alt_values = calc_nodes_alt_values(agent_num, task_num, tasks, tree_info, a_taskInds, allocation_structure, task_cons, realtime_node_values, temp_node_values)


    re_assignment_count = 0
    iteration_count = 0
    while True:
        iteration_count += 1

        feasible_choices = []

        match (greedy_level):
            case 2:
                if not (best_move_NE[0] > 0 or (best_move_NE[0] == 0 and (best_move_NE[1] > 0 or (best_move_NE[1] == 0 and best_move_NE[2] != task_num and allocation_structure[best_move_NE[3]] == task_num)))): 
                    break  # reach NE solution

                selected_a_index = best_move_NE[3]                
                new_t_index = best_move_NE[2]
                sys_improvement_value = best_move_NE[0]
                task_movement_value = best_move_NE[1]

            case 1:
                feasible_choices = [i for i in agentIds if max_moves[i][0] > 0 or (max_moves[i][0] == 0 and (max_moves[i][1] > 0 or (max_moves[i][1] == 0 and max_moves[i][2] != task_num and allocation_structure[i] == task_num)))]

                if len(feasible_choices) == 0:
                    break  # reach NE solution
                    # when eps = 1, it's Random, when eps = 0, it's Greedy
                if np.random.uniform() <= eps:
                    # exploration: random allocation
                    selected_a_index = np.random.choice(feasible_choices)
                else:
                    # exploitation: allocation else based on reputation or efficiency
                    # selected_a_index = np.argmax(max_moves)
                    selected_a_index = max(enumerate(max_moves), key=lambda x: (x[1][0], x[1][1], x[1][2] != task_num and allocation_structure[x[0]] == task_num))[0]
                    
                sys_improvement_value = max_moves[selected_a_index][0]
                task_movement_value = max_moves[selected_a_index][1]
                new_t_index = max_moves[selected_a_index][2]

            case _:
                feasible_choices = [[] for i in range(0, agent_num)]
                for i in agentIds:
                    a_cur_t = allocation_structure[i]
                    for j in a_taskInds[i]:
                        root_improvement = node_alt_values[i][j][root_node_index] - realtime_node_values[root_node_index]
                        task_move_val_i_j = task_cons[i][j] - cur_con[i]
                        if root_improvement > 0:
                            feasible_choices[i].append((root_improvement, task_move_val_i_j, j, a_cur_t, i))
                            continue
                        if root_improvement == 0:
                            if task_move_val_i_j > 0:
                                feasible_choices[i].append((root_improvement, task_move_val_i_j, j, a_cur_t, i))
                                continue
                            if task_move_val_i_j == 0 and a_cur_t == task_num:
                                feasible_choices[i].append((root_improvement, task_move_val_i_j, j, a_cur_t, i))
                                continue

                if all([len(feasible_choices[i]) == 0 for i in range(0, agent_num)]):
                    break  # reach NE solution
                if np.random.uniform() <= eps:
                    # exploration: random allocation
                    selected_a_index = random.choice([i for i in agentIds if len(feasible_choices[i]) > 0])
                    new_t_index = random.choice(feasible_choices[selected_a_index])[2]
                else:
                    best_move = (float("-inf"), float("-inf"), 0, 0, 0)
                    for i in agentIds:
                        a_cur_t = allocation_structure[i]
                        if len(feasible_choices[i]) == 0:
                            continue
                        best_move_i = max(feasible_choices[i], key=lambda x: (x[0], x[1], x[2] != task_num and a_cur_t == task_num))
                        if best_move_i[0] > best_move[0]:
                            best_move = best_move_i
                            continue
                        if best_move_i[0] == best_move[0]:
                            if best_move_i[1] > best_move[1]:
                                best_move = best_move_i
                                continue
                            if best_move_i[1] == best_move[1] and a_cur_t == task_num and best_move[3] != task_num:
                                best_move = best_move_i
                                continue
                    selected_a_index = best_move[4]
                    new_t_index = best_move[2]
                
                sys_improvement_value = node_alt_values[selected_a_index][new_t_index][root_node_index] - realtime_node_values[root_node_index]
                task_movement_value = task_cons[selected_a_index][new_t_index] - cur_con[selected_a_index]


        # perform move
        old_t_index = allocation_structure[selected_a_index]
        allocation_structure[selected_a_index] = new_t_index
        coalition_structure[new_t_index].append(selected_a_index)
        coalition_structure[old_t_index].remove(selected_a_index)

        # print(
        #     f"[{iteration_count}]", 
        #     f"\tAgent:{selected_a_index}",  
        #     f"\tOld Task:{old_t_index}",  
        #     f"\tNew Task:{new_t_index}", 
        #     f"\tCur Con:{cur_con[selected_a_index]}",
        #     f"\tOld TCons:{task_cons[selected_a_index][old_t_index]}", 
        #     f"\tNew TCons:{task_cons[selected_a_index][new_t_index]}", 
        #     f"\tSys:{sys_improvement_value}",
        #     f"\tMove:{task_movement_value}",
        # )
        # if iteration_count % 100 == 0:
        #     pass

        # update agents in the new coalition
        affected_t_indexes = []
        if new_t_index != task_num:
            affected_t_indexes.append(new_t_index)
            for i in coalition_structure[new_t_index]:
                task_cons[i][new_t_index] = agent_contribution(agents, tasks, i, new_t_index, coalition_structure[new_t_index], constraints, gamma)
                cur_con[i] = task_cons[i][new_t_index]
        else:
            task_cons[selected_a_index][new_t_index] = 0
            cur_con[selected_a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != task_num):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assignment_count += 1
            affected_t_indexes.append(old_t_index)
            for i in coalition_structure[old_t_index]:
                task_cons[i][old_t_index] = agent_contribution(agents, tasks, i, old_t_index, coalition_structure[old_t_index], constraints, gamma)
                cur_con[i] = task_cons[i][old_t_index]

        # update other agents with respect to the affected tasks
        for t_ind in affected_t_indexes:
            for i in agentIds:
                if (t_ind in a_taskInds[i]) and (i not in coalition_structure[t_ind]):
                    task_cons[i][t_ind] = agent_contribution(agents, tasks, i, t_ind, coalition_structure[t_ind], constraints, gamma)
                

        # Update real-time node values
                    
        if greedy_level == 2 or greedy_level == 1:

            # First, update the node values when the chosen agent is removed from the system
            realtime_node_values = temp_node_values[selected_a_index].copy()

            # Second, update the node values when the chosen agent is added to the new coalition
            realtime_node_values[new_t_index] += cur_con[selected_a_index]
            sys_added_value = cur_con[selected_a_index]
            node_val = realtime_node_values[new_t_index]
            parent_id = tree_info[new_t_index].parent_id

            while parent_id is not None and parent_id != len(tasks) and sys_added_value > 0:
                parent_val = realtime_node_values[parent_id]

                if(tree_info[parent_id].node_type == NodeType.AND):
                    realtime_node_values[parent_id] += sys_added_value
                
                elif(tree_info[parent_id].node_type == NodeType.OR):
                    if (parent_val < node_val):
                        sys_added_value = node_val - parent_val
                        realtime_node_values[parent_id] = node_val
                    else:
                        sys_added_value = 0
                        break

                node_val = realtime_node_values[parent_id]
                if parent_id == root_node_index:
                    break
                parent_id = tree_info[parent_id].parent_id

        else:
            realtime_node_values = node_alt_values[selected_a_index][new_t_index].copy()

        
        # For each agent, recalculate temp_node_values and the max move value
        
        for i in agentIds:
            if (i != selected_a_index):
                # We can skip calculating the temp_node_values of the selected agent, since it's just been updated
                temp_node_values[i] = calc_temp_node_values(i, tasks, tree_info, allocation_structure, cur_con, realtime_node_values, root_node_index=root_node_index)
                
        
        match (greedy_level):
            case 2:
                best_move_NE = calc_best_move(agent_num, task_num, tasks, tree_info, a_taskInds, allocation_structure, cur_con, task_cons, realtime_node_values, temp_node_values, root_node_index=root_node_index)
            case 1:
                max_moves = calc_max_move_value(agent_num, task_num, tasks, tree_info, a_taskInds, allocation_structure, cur_con, task_cons, realtime_node_values, temp_node_values, root_node_index=root_node_index)
            case _:
                node_alt_values = calc_nodes_alt_values(agent_num, task_num, tasks, tree_info, a_taskInds, allocation_structure, task_cons, realtime_node_values, temp_node_values)
                

    return (
        coalition_structure,
        realtime_node_values[root_node_index],
        iteration_count,
        re_assignment_count,
    )



In [ ]:
def task_reward(task, agents, gamma=1):
    # task is represented by a list of capabilities it requires, agents is a list agents, where each represented by a list cap contribution values
    """
    Calculate the reward of a single task
    :param: `task`: the list of capabilities the task requires
    :param: `agents`: the list of agents
    :param: `gamma`: the discount factor
    :return: the reward of the task
    """
    if agents == []:
        return 0
    else:
        return sum([max([agent[c] for agent in agents]) for c in task]) * (
            gamma ** len(agents)
        )

def sys_rewards_tree_agents(tree_info : list[Node], tasks, agents, allocation_structure, root_node_index=-1, gamma=1):
    """
    Calculate the reward of the system, given the allocation structure: agent -> task
    """
    def sys_rewards_node(node : Node):
        if node.node_type == NodeType.LEAF or node.node_type == NodeType.DUMMY:
            return task_reward(tasks[node.node_id], [agent for i, agent in enumerate(agents) if allocation_structure[i] == node.node_id], gamma)
        rewards = [sys_rewards_node(tree_info[i]) for i in node.children_ids]
        if node.node_type == NodeType.AND:
            return sum(rewards)
        elif node.node_type == NodeType.OR:
            return max(rewards)
    return sys_rewards_node(tree_info[root_node_index])


def sys_rewards_tree_tasks(tree_info, tasks, agents, coalition_structure, root_node_index=-1, gamma=1):
    """
    Calculate the reward of the system, given the coalition structure: task -> agents (coalition)
    """
    def sys_rewards_node(node : Node):
        if node.node_type == NodeType.LEAF or node.node_type == NodeType.DUMMY:
            return task_reward(tasks[node.node_id], [agents[i] for i in coalition_structure[node.node_id]], gamma)
        rewards = [sys_rewards_node(tree_info[i]) for i in node.children_ids]
        if node.node_type == NodeType.AND:
            return sum(rewards)
        elif node.node_type == NodeType.OR:
            return max(rewards)
    return sys_rewards_node(tree_info[root_node_index])


In [ ]:
def random_solution_and_or_tree(agents, tasks, constraints, tree_info: list[Node], gamma=1):
    '''
    Randomly allocate tasks to agents
    '''
    task_num = len(tasks)
    agent_num = len(agents)
    a_taskInds = constraints[0]
    allocation_structure = [np.random.choice(a_taskInds[i] + [task_num]) for i in range(0, agent_num)]
    return allocation_structure, sys_rewards_tree_agents(tree_info, tasks, agents, allocation_structure, gamma=gamma)


In [ ]:

class GreedyNE:
    def __init__(self, agents, tasks, constraints, tree_info : list[Node], root_node_index=-1, gamma=1, coalition_structure=[], greedy_level=2):
        self.agents : list[list[float]] = agents
        self.tasks : list[list[float]] = tasks
        self.constraints = constraints
        self.tree_info = tree_info
        self.root_node_index = root_node_index
        self.gamma = gamma
        self.greedy_level = greedy_level
        self.dummy_task_id = len(tasks)

        self.task_num = len(tasks)
        self.agent_num = len(agents)
        self.agentIds = list(range(0, self.agent_num))
        self.a_taskInds : list[list[int]] = constraints[0]

        self.realtime_node_values = [0 for node in range(0, len(tree_info))]
        self.temp_node_values = {i : {node_id : node_value for node_id, node_value in enumerate(self.realtime_node_values)} for i in self.agentIds}

        if self.root_node_index < 0:
            self.root_node_index = len(tree_info) + self.root_node_index

        if coalition_structure == None or coalition_structure == []:
            self.coalition_structure = [[] for j in range(0, self.task_num)] + [list(range(0, self.agent_num))]
            self.allocation_structure = [self.task_num for i in range(0, self.agent_num)]
        else:
            self.coalition_structure = coalition_structure
            for j in range(0, self.task_num + 1):
                for i in coalition_structure[j]:
                    self.allocation_structure[i] = j

        self.cur_con = [
            self.get_agent_contribution(i, j)
            for i, j in enumerate(self.allocation_structure)
        ]

        self.task_cons = [
            [
                self.get_agent_contribution(i, j)
                if j in self.a_taskInds[i]
                else float("-inf")
                for j in range(0, self.task_num)
            ] + [0]
            for i in range(0, self.agent_num)
        ]

    def get_agent_contribution(self, query_agentIndex, query_taskIndex):
        """
        Return contribution of agent i to task j in coalition C_j
        
        = U_i(C_j, j) - U_i(C_j \ {i}, j) if i in C_j

        = U_i(C_j U {i}, j) - U_i(S, j) if i not in C_j
        """
        if query_taskIndex == self.dummy_task_id:
            return 0
        if query_taskIndex not in self.a_taskInds[query_agentIndex]:
            return 0
        cur_reward = task_reward(self.tasks[query_taskIndex], [self.agents[i] for i in self.coalition_structure[query_taskIndex]], self.gamma)
        coalition = self.coalition_structure[query_taskIndex]
        if self.allocation_structure[query_agentIndex] == query_taskIndex:
            return cur_reward - task_reward(self.tasks[query_taskIndex], [self.agents[i] for i in coalition if i != query_agentIndex], self.gamma)
        else:
            return task_reward(self.tasks[query_taskIndex], [self.agents[i] for i in coalition] + [self.agents[query_agentIndex]], self.gamma) - cur_reward
        
    def calc_temp_node_values(self, query_a_index, parent_info, children_info, root_node_index=-1):
        """
        Calculate temp_node_values[i], for when agent i is removed from the system
        """
        temp_node_values_i = {}
        
        node_id : int = self.allocation_structure[query_a_index]

        sys_lost_value = self.cur_con[query_a_index]

        temp_node_values_i[node_id] = self.realtime_node_values[node_id] - sys_lost_value
        
        prev_node_value = temp_node_values_i[node_id]
        prev_node_id = node_id
        node_id = parent_info[node_id]

        while sys_lost_value > 0:

            if self.tree_info[node_id].node_type == NodeType.AND:
                temp_node_values_i[node_id] = self.realtime_node_values[node_id] - sys_lost_value
            elif self.tree_info[node_id].node_type == NodeType.OR:
                max_node_value = max([self.realtime_node_values[j] for j in children_info[node_id] if j != prev_node_id] + [prev_node_value])
                temp_node_values_i[node_id] = max_node_value
                sys_lost_value = self.realtime_node_values[node_id] - max_node_value

            if node_id == root_node_index:
                break

            prev_node_value = temp_node_values_i[node_id]
            prev_node_id = node_id
            node_id = parent_info[node_id]

        return temp_node_values_i
    
    
    def update_coalitions(self, selected_a_index, old_t_index, new_t_index):
        """
        Move agent i to coalition j
        """

        # perform move
        self.allocation_structure[selected_a_index] = new_t_index
        self.coalition_structure[new_t_index].append(selected_a_index)
        self.coalition_structure[old_t_index].remove(selected_a_index)
    
    def update_tasks_contribution_values(self, selected_a_index, old_t_index, new_t_index):
        """
        Update the contribution values of agent i to each task
        """

        re_assigned = 0

        # update agents in the new coalition
        affected_t_indexes = []
        if new_t_index != self.dummy_task_id:
            affected_t_indexes.append(new_t_index)
            for i in self.coalition_structure[new_t_index]:
                self.task_cons[i][new_t_index] = self.get_agent_contribution(i, new_t_index)
                self.cur_con[i] = self.task_cons[i][new_t_index]
        else:
            self.task_cons[selected_a_index][new_t_index] = 0
            self.cur_con[selected_a_index] = 0

        # update agent in the old coalition (if applicable)
        if (old_t_index != self.dummy_task_id):  
            # if agents indeed moved from another task, we have to change every agent from the old as well
            re_assigned = 1
            affected_t_indexes.append(old_t_index)
            for i in self.coalition_structure[old_t_index]:
                self.task_cons[i][old_t_index] = self.get_agent_contribution(i, old_t_index)
                self.cur_con[i] = self.task_cons[i][old_t_index]

        # update other agents with respect to the affected tasks
        for t_ind in affected_t_indexes:
            for i in self.agentIds:
                if (t_ind in self.a_taskInds[i]) and (i not in self.coalition_structure[t_ind]):
                    self.task_cons[i][t_ind] = self.get_agent_contribution(i, t_ind)

        return re_assigned


    def get_temp_node_value(self, query_node_id, temp_node_values : dict):
        """
        Return the temp_node_value of agent i at node j
        """
        return temp_node_values.get(query_node_id, self.realtime_node_values[query_node_id]) 


    def get_best_move(self, temp_node_values : dict[int, dict[int]], root_node_index=-1):
        """
        Calculate the best move values among all agents
        """
        best_sys_move_value, best_move_value, best_move_coalition, best_move_agent, best_move_node_revalue_sequence = float("-inf"), float("-inf"), self.allocation_structure[0], 0, {}
        
        for i in self.agentIds:
            # Movement value for moving agent from current coalition to dummy coalition (removing the agent from the system):
            sys_exit_value = temp_node_values[i].get(root_node_index, self.realtime_node_values[root_node_index]) - self.realtime_node_values[root_node_index]
            
            # Calculate the best move values for each agent
            for j in self.a_taskInds[i]:

                if j == self.allocation_structure[i]:
                    continue
            
                sys_added_value = self.task_cons[i][j]
                node_revalue_sequence = {}

                node_revalue_sequence[j] = temp_node_values[i].get(j, self.realtime_node_values[j]) + sys_added_value
                prev_node_id = j
                node_id = self.tree_info[j].parent_id

                # Backtrack to the root node
                while (sys_added_value + sys_exit_value) >= best_sys_move_value and sys_added_value > 0 and prev_node_id != root_node_index:

                    prev_node_val = node_revalue_sequence[prev_node_id]
                    node_val = temp_node_values[node_id].get(node_id, self.realtime_node_values[node_id])

                    if(self.tree_info[node_id].node_type == NodeType.AND):
                        node_revalue_sequence[node_id] = node_val + sys_added_value
                    
                    elif(self.tree_info[node_id].node_type == NodeType.OR):
                        node_revalue_sequence[node_id] = max(node_val, prev_node_val)
                        sys_added_value = node_revalue_sequence[node_id] - node_val

                    prev_node_id = node_id
                    node_id = self.tree_info[node_id].parent_id
                
                # Compare the calculated system move value with the best move value

                sys_move_val_i_j = sys_exit_value + sys_added_value
                task_move_val_i_j = self.task_cons[i][j] - self.cur_con[i]

                if(sys_move_val_i_j < best_sys_move_value):
                    continue

                if sys_move_val_i_j > best_sys_move_value:
                    best_sys_move_value, best_move_value, best_move_coalition, best_move_agent, best_move_node_revalue_sequence = sys_move_val_i_j, task_move_val_i_j, j, i, node_revalue_sequence
                    continue     
                
                # else: sys_move_val_i_j == best_move[0]:
                
                # Tie breaking: choose the one with higher contribution to a single task
                if task_move_val_i_j > best_move_value:
                    best_sys_move_value, best_move_value, best_move_coalition, best_move_agent, best_move_node_revalue_sequence = sys_move_val_i_j, task_move_val_i_j, j, i, node_revalue_sequence
                    continue
                
                # Tie breaking: choose the one that moves out of the dummy coalition
                if task_move_val_i_j == best_move_value and self.allocation_structure[i] == self.dummy_task_id:
                    best_sys_move_value, best_move_value, best_move_coalition, best_move_agent, best_move_node_revalue_sequence = sys_move_val_i_j, task_move_val_i_j, j, i, node_revalue_sequence
                    continue
            
        return best_sys_move_value, best_move_value, best_move_coalition, best_move_agent, best_move_node_revalue_sequence


    def solve(self):

        parent_info = {node.node_id: node.parent_id for node in self.tree_info}
        children_info = {node.node_id: node.children_ids.copy() for node in self.tree_info}

        temp_node_values = {i : self.calc_temp_node_values(i, parent_info, children_info, root_node_index=self.root_node_index) for i in self.agentIds}

        iteration_count = 0
        re_assignment_count = 0
        while True:
            iteration_count += 1

            sys_move_val, task_move_val, new_t_id, selected_a_id, node_revalue_sequence = self.get_best_move(temp_node_values, root_node_index=self.root_node_index)

            if sys_move_val < 0:
                break
            elif sys_move_val == 0:
                if task_move_val < 0:
                    break
                elif task_move_val == 0:
                    if self.allocation_structure[selected_a_id] != self.dummy_task_id:
                        break

            old_t_id = self.allocation_structure[selected_a_id]

            self.update_coalitions(selected_a_id, old_t_id, new_t_id)

            re_assigned = self.update_tasks_contribution_values(selected_a_id, old_t_id, new_t_id)
            
            re_assignment_count += re_assigned

            # Update real-time node values

            for node_id, node_value in temp_node_values[selected_a_id].items():
                self.realtime_node_values[node_id] = node_value

            for node_id, node_value in node_revalue_sequence.items():
                self.realtime_node_values[node_id] = node_value
            
            # For each agent, recalculate temp_node_values and the max move value
            
            temp_node_values = {i : self.calc_temp_node_values(i, parent_info, children_info, root_node_index=self.root_node_index) for i in self.agentIds}

            for i in self.agentIds:
                if (i != selected_a_id):
                    # We can skip calculating the temp_node_values of the selected agent, since it's just been updated
                    temp_node_values[i] = self.calc_temp_node_values(i, parent_info, children_info, root_node_index=self.root_node_index)

        return self.coalition_structure, self.realtime_node_values[self.root_node_index], iteration_count, re_assignment_count


In [ ]:

def append_record(record, filename, typ):
    with open(filename, "a") as f:
        if typ != "":
            json.dump(record, f, default=typ)
        else:
            json.dump(record, f)
        f.write(os.linesep)
        f.close()

def main():
    run_num = 1000
    gamma = 1
    a_min_edge = 1

    capNum = 10
    max_capVal = 10
    max_capNum_task = 10
    max_capNum_agent = 10
    ex_identifier = 0
    time_bound = 600
    task_num = 100
    agent_num = 200

    capabilities = list(range(0, capNum))

    for run in range(0, run_num):
        t_max_edge = 3
        while t_max_edge <= 50:
            ex_identifier += 1

            #         agent_num = np.random.randint(task_num,3*task_num)
            tasks = gen_tasks(task_num, max_capNum_task, capabilities)
            constraints = gen_constraints(
                agent_num, task_num, 1, a_min_edge, t_max_edge
            )
            a_taskInds = constraints[0]
            agents_cap, agents = gen_agents(
                a_taskInds, tasks, max_capNum_agent, capabilities, max_capVal
            )
            tree_info = gen_tree_info(task_num=task_num)
            # num_com = np.prod([1 if a_taskInds[i] == [] else len(a_taskInds[i])+1 for i in range(0,agent_num)])

            num_com = reduce(
                mul,
                [
                    1 if a_taskInds[i] == [] else len(a_taskInds[i]) + 1
                    for i in range(0, agent_num)
                ],
            )

            up = upperBound(capabilities, tasks, agents)

            up2 = upperBound_ver2(capabilities, tasks, agents, constraints)
            print("UP:", up, "  UP2:", up2)

            result = {
                "ex_identifier": ex_identifier,
                "task_num": task_num,
                "agent_num": agent_num,
                "up": up,
                "up2": up2,
            }
            #         data = {"ex_identifier":ex_identifier,"tasks":tasks,"constraints":constraints,"agents_cap":agents_cap,"agents":agents}

            a_den = [len(c) for c in constraints[0]]
            t_den = [len(c) for c in constraints[1]]
            result["a_den_avg"] = statistics.mean(a_den)
            result["a_den_max"] = max(a_den)
            result["a_den_min"] = min(a_den)

            result["t_den_avg"] = statistics.mean(t_den)
            result["t_den_max"] = max(t_den)
            result["t_den_min"] = min(t_den)
            result["t_max_edge"] = t_max_edge

            print(
                "density", t_max_edge, "task_num:", task_num, "  agent_num:", agent_num
            )
            print(
                "a_den_avg",
                result["a_den_avg"],
                "a_den_max:",
                result["a_den_max"],
                "  a_den_min:",
                result["a_den_min"],
            )

            print(
                "t_den_avg",
                result["t_den_avg"],
                "t_den_max:",
                result["t_den_max"],
                "  t_den_min:",
                result["t_den_min"],
            )
            print("-----------------------------------")
            print("Heterogeneous Tasks")
            print("-----------------------------------")

            start = time.perf_counter()
            r = eGreedy2(agents, tasks, constraints, gamma=gamma)
            end = time.perf_counter()
            result["g"] = r[1]
            result["g_iter"] = r[2]
            result["g_reass"] = r[3]
            result["g_t"] = end - start
            print(
                "eGreedy:",
                "\ttime:",
                result["g_t"],
                "\tresult:",
                result["g"],
                "\titeration:",
                result["g_iter"],
                "\tre-assignment",
                result["g_reass"],
            )

            start = time.perf_counter()
            rand_sol_a, rand_sol_reward = random_solution_heterogeneous(agents, tasks, constraints, gamma=gamma)
            end = time.perf_counter()
            print(
                "Random Solution:",
                "\ttime:",
                end - start,
                "\tresult:",
                rand_sol_reward,
            )

            print("-----------------------------------")
            print("AND-OR Goal Tree Tasks")
            print("-----------------------------------")

            nodes_constraints = get_nodes_constraints(tree_info, constraints, root_node_index=-1)

            start = time.perf_counter()
            up_tree_root = upperBoundTree_root(tree_info, capabilities, tasks, agents)
            end = time.perf_counter()

            print("UP Tree:", up_tree_root, "\ttime:", end - start)

            start = time.perf_counter()
            up_tree_1 = upperBoundTree_allNodes_v1(tree_info, capabilities, tasks, agents, nodes_constraints)
            end = time.perf_counter()

            print("UP1:", up_tree_1, "\ttime:", end - start)

            start = time.perf_counter()
            up_tree_2 = upperBoundTree_allNodes_v2(tree_info, capabilities, tasks, agents, nodes_constraints)
            end = time.perf_counter()

            print("UP2:", up_tree_2, "\ttime:", end - start)

            start = time.perf_counter()
            r_tree_solver = GreedyNE(agents, tasks, constraints, tree_info=tree_info, gamma=gamma)
            result_c = r_tree_solver.solve()
            end = time.perf_counter()
            print(
                "GreedyNE Class:",
                "\ttime:",
                end - start,
                "\tresult:",
                result_c[1],
                "\titeration:",
                result_c[2],
                "\tre-assignment",
                result_c[3],
            )

            start = time.perf_counter()
            r_tree_0 = greedyNETree(agents, tasks, constraints, tree_info=tree_info, gamma=gamma, greedy_level=0)
            end = time.perf_counter()
            print(
                "GreedyNE 0:",
                "\ttime:",
                end - start,
                "\tresult:",
                r_tree_0[1],
                "\titeration:",
                r_tree_0[2],
                "\tre-assignment",
                r_tree_0[3],
            )

            start = time.perf_counter()
            r_tree_1 = greedyNETree(agents, tasks, constraints, tree_info=tree_info, gamma=gamma, greedy_level=1)
            end = time.perf_counter()
            print(
                "GreedyNE 1:",
                "\ttime:",
                end - start,
                "\tresult:",
                r_tree_1[1],
                "\titeration:",
                r_tree_1[2],
                "\tre-assignment",
                r_tree_1[3],
            )
            start = time.perf_counter()
            r_tree_2 = greedyNETree(agents, tasks, constraints, tree_info=tree_info, gamma=gamma, greedy_level=2)
            end = time.perf_counter()
            print(
                "GreedyNE 2:",
                "\ttime:",
                end - start,
                "\tresult:",
                r_tree_2[1],
                "\titeration:",
                r_tree_2[2],
                "\tre-assignment",
                r_tree_2[3],
            )
            start = time.perf_counter()
            r_tree_0 = greedyNETree(agents, tasks, constraints, tree_info=tree_info, gamma=gamma, greedy_level=0, eps=1)
            end = time.perf_counter()
            print(
                "GreedyNE 0, eps=1:",
                "\ttime:",
                end - start,
                "\tresult:",
                r_tree_0[1],
                "\titeration:",
                r_tree_0[2],
                "\tre-assignment",
                r_tree_0[3],
            )

            start = time.perf_counter()
            r_tree_1 = greedyNETree(agents, tasks, constraints, tree_info=tree_info, gamma=gamma, greedy_level=1, eps=1)
            end = time.perf_counter()
            print(
                "GreedyNE 1, eps=1:",
                "\ttime:",
                end - start,
                "\tresult:",
                r_tree_1[1],
                "\titeration:",
                r_tree_1[2],
                "\tre-assignment",
                r_tree_1[3],
            )
            start = time.perf_counter()
            rand_sol_a, rand_sol_reward = random_solution_and_or_tree(agents, tasks, constraints, tree_info=tree_info, gamma=gamma)
            end = time.perf_counter()
            print(
                "Random Solution:",
                "\ttime:",
                end - start,
                "\tresult:",
                rand_sol_reward,
            )

            print("-----------------------------------")
            print("Heterogeneous Tasks (OPD, FMS)")
            print("-----------------------------------")

            if t_max_edge < 15:
                start = time.perf_counter()
                opd = OPD(agents, tasks, constraints, gamma)
                new_con = opd[0:2]
                result["OPD_t"] = time.perf_counter() - start
                r = FMS(agents, tasks, new_con, gamma=gamma, time_bound=time_bound)
                end = time.perf_counter()
                result["OPD_pruned"] = opd[2]
                print(
                    "OPD time used:",
                    result["OPD_t"],
                    " Edge pruned:",
                    result["OPD_pruned"],
                )
                result["BnBFMS"] = r[1]
                result["BnBFMS_iter"] = r[2]
                result["BnBFMS_t"] = end - start
                result["BnBFMS_converge"] = r[4]
                print(
                    "BnB FMS time:",
                    result["BnBFMS_t"],
                    "result:",
                    result["BnBFMS"],
                    "iteration:",
                    result["BnBFMS_iter"],
                    "converge?",
                    result["BnBFMS_converge"],
                )
            print()

            # # append data and result
            # files = {"density100_result_cap": [result, ""]}

            # for filename in list(files.keys()):
            #     append_record(files[filename][0], filename, typ=files[filename][1])

            # increase the task_num
            t_max_edge += 1
        
        break


if __name__ == "__main__":
    main()